In [26]:
import re
import numpy as np
import plotly.graph_objects as go
from scipy.spatial import ConvexHull

In [2]:
def parse_exf(file_path):
    """
    Parses the .exf file and extracts the first coordinate (x, y, z) for each node.
    For node templates (e.g., node1, node2, node3) that span multiple lines,
    the script assumes that each node block starts with a line "Node: <number>"
    and that the next three lines each contain numbers where the first value is the coordinate.
    """
    nodes = []
    with open(file_path, 'r') as f:
        lines = f.readlines()

    i = 0
    while i < len(lines):
        line = lines[i].strip()
        # Detect a node definition line
        if line.startswith("Node:"):
            # Extract node number (optional)
            match = re.match(r"Node:\s*(\d+)", line)
            node_number = int(match.group(1)) if match else None

            coords = []
            # Expecting the next 3 lines to contain the x, y, and z coordinates
            # (each line starts with the value, followed by derivative info)
            for j in range(1, 4):
                if i + j < len(lines):
                    coord_line = lines[i + j].strip()
                    # Split the line into parts and take the first value as coordinate
                    parts = coord_line.split()
                    if parts:
                        try:
                            val = float(parts[0])
                            coords.append(val)
                        except ValueError:
                            # If conversion fails, skip this node.
                            break
            if len(coords) == 3:
                nodes.append({
                    'node': node_number,
                    'x': coords[0],
                    'y': coords[1],
                    'z': coords[2]
                })
            i += 4  # Move past the current node block (Node line + 3 coordinate lines)
        else:
            i += 1

    return nodes

In [3]:
# Replace with the correct path to your file
file_path = "data/Geometry_Fitter_Wholebody.exf"
nodes = parse_exf(file_path)
print("Total nodes parsed:", len(nodes))


Total nodes parsed: 4027


In [20]:
# Extract the node IDs (filtering out any that might be None)
node_ids = [node['node'] for node in nodes if node['node'] is not None]

# Create a histogram to show the distribution of node IDs
fig_hist = go.Figure(data=[go.Histogram(x=node_ids, nbinsx=50)])
fig_hist.update_layout(
    title="Histogram of Node IDs",
    xaxis_title="Node ID",
    yaxis_title="Count",
    width=800,   # Adjust the width as needed
    height=600   # Adjust the height as needed
)
fig_hist.show()

In [21]:
# Separate nodes into two groups based on their node id
group1 = [node for node in nodes if node['node'] < 7000]
group2 = [node for node in nodes if node['node'] >= 7000]


In [25]:
# Extract coordinate lists for plotting
x_vals = [node['x'] for node in nodes]
y_vals = [node['y'] for node in nodes]
z_vals = [node['z'] for node in nodes]

# Compute a unified axis range using the min and max of all coordinate values
global_min = min(min(x_vals), min(y_vals), min(z_vals))
global_max = max(max(x_vals), max(y_vals), max(z_vals))
abs_max = max(abs(global_min), abs(global_max))
axis_range = [-abs_max, abs_max]

# Create a 3D scatter plot using Plotly
fig = go.Figure(data=[go.Scatter3d(
    x=x_vals,
    y=y_vals,
    z=z_vals,
    mode='markers',
    marker=dict(size=1, color="black")
)])
fig.update_layout(
    title='3D Scatter Plot of SPARC Human Wholebody Scaffold',
    scene=dict(
        xaxis=dict(title='X', range=axis_range),
        yaxis=dict(title='Y', range=axis_range),
        zaxis=dict(title='Z', range=[range + 800 for range in axis_range]),
        aspectmode='cube'  # Ensures equal scale for x, y, z axes
    ),
    width=800,   # Increase width as needed
    height=800   # Increase height as needed
)

fig.show()

In [22]:
# Extract coordinate lists for plotting
x_vals = [node['x'] for node in group1]
y_vals = [node['y'] for node in group1]
z_vals = [node['z'] for node in group1]

# Compute a unified axis range using the min and max of all coordinate values
global_min = min(min(x_vals), min(y_vals), min(z_vals))
global_max = max(max(x_vals), max(y_vals), max(z_vals))
abs_max = max(abs(global_min), abs(global_max))
axis_range = [-abs_max, abs_max]

# Create a 3D scatter plot using Plotly
fig = go.Figure(data=[go.Scatter3d(
    x=x_vals,
    y=y_vals,
    z=z_vals,
    mode='markers',
    marker=dict(size=2)
)])
fig.update_layout(
    title='3D Scatter Plot of SPARC Human Wholebody Scaffold',
    scene=dict(
        xaxis=dict(title='X', range=axis_range),
        yaxis=dict(title='Y', range=axis_range),
        zaxis=dict(title='Z', range=[range + 800 for range in axis_range]),
        aspectmode='cube'  # Ensures equal scale for x, y, z axes
    ),
    width=800,   # Increase width as needed
    height=800   # Increase height as needed
)

fig.show()

In [23]:
# Extract coordinate lists for plotting
x_vals = [node['x'] for node in group2]
y_vals = [node['y'] for node in group2]
z_vals = [node['z'] for node in group2]

# Compute a unified axis range using the min and max of all coordinate values
global_min = min(min(x_vals), min(y_vals), min(z_vals))
global_max = max(max(x_vals), max(y_vals), max(z_vals))
abs_max = max(abs(global_min), abs(global_max))
axis_range = [-abs_max, abs_max]

# Create a 3D scatter plot using Plotly
fig = go.Figure(data=[go.Scatter3d(
    x=x_vals,
    y=y_vals,
    z=z_vals,
    mode='markers',
    marker=dict(size=2)
)])
fig.update_layout(
    title='3D Scatter Plot of SPARC Human Wholebody Scaffold',
    scene=dict(
        xaxis=dict(title='X', range=axis_range),
        yaxis=dict(title='Y', range=axis_range),
        zaxis=dict(title='Z', range=[range + 800 for range in axis_range]),
        aspectmode='cube'  # Ensures equal scale for x, y, z axes
    ),
    width=800,   # Increase width as needed
    height=800   # Increase height as needed
)

fig.show()

# Plotting mesh instead of points

In [32]:
def parse_mesh3d_elements(file_path):
    """
    Parses the exf file to extract connectivity for the 3D mesh (element3 section).
    
    This function looks for the mesh3d section (marked by "!#mesh mesh3d") and then for each
    element in that section it extracts the element number and the list of eight node IDs
    defined after the "Nodes:" line.
    
    Returns:
        A list of dictionaries, each with keys:
          'element': element ID (int)
          'nodes': list of 8 node IDs (ints)
    """
    with open(file_path, 'r') as f:
        lines = f.readlines()

    elements = []
    inside_mesh3d = False
    i = 0
    while i < len(lines):
        line = lines[i].strip()
        # Look for the start of the mesh3d section
        if line.startswith("!#mesh mesh3d"):
            inside_mesh3d = True
            # Move to the next line to start processing details in mesh3d
            i += 1
            continue

        # Once inside mesh3d, look for elements
        if inside_mesh3d and line.startswith("Element:"):
            match = re.match(r"Element:\s*(\d+)", line)
            if match:
                elem_id = int(match.group(1))
                connectivity = []
                # Advance through lines until we find the "Nodes:" label
                found_nodes = False
                while i < len(lines):
                    i += 1
                    if i >= len(lines):
                        break
                    subline = lines[i].strip()
                    if subline.startswith("Nodes:"):
                        found_nodes = True
                        parts = subline.split()
                        # If the "Nodes:" line contains numbers on the same line,
                        # they will be after the label.
                        if len(parts) > 1:
                            connectivity = [int(x) for x in parts[1:]]
                        else:
                            # Otherwise, assume the next line contains the node IDs.
                            if i + 1 < len(lines):
                                i += 1
                                connectivity = [int(x) for x in lines[i].strip().split()]
                        break  # Exit loop after processing nodes
                if found_nodes and len(connectivity) == 8:
                    elements.append({
                        'element': elem_id,
                        'nodes': connectivity
                    })
            else:
                i += 1
        else:
            i += 1

    return elements

In [33]:
file_path = "data/Geometry_Fitter_Wholebody.exf"  # Replace with the path to your file
elements = parse_mesh3d_elements(file_path)
print("Total 3D elements parsed:", len(elements))
# Show the first few parsed elements
for elem in elements[:5]:
    print(elem)

Total 3D elements parsed: 1276
{'element': 1, 'nodes': [1, 4, 54, 57, 2, 5, 55, 58]}
{'element': 2, 'nodes': [2, 5, 55, 58, 3, 6, 56, 59]}
{'element': 3, 'nodes': [4, 7, 57, 60, 5, 8, 58, 61]}
{'element': 4, 'nodes': [5, 8, 58, 61, 6, 9, 59, 62]}
{'element': 5, 'nodes': [7, 10, 60, 63, 8, 11, 61, 64]}


In [37]:
# Example function to convert a hexahedral element (8 nodes) into triangle faces.
def hexahedron_to_triangles(hex_nodes):
    """
    Given a list of 8 node indices (assumed ordering: bottom face nodes first, then top face nodes),
    returns a list of triangles (each triangle is a list of 3 node indices) that represent the hexahedron's faces.
    """
    # Define faces as quadrilaterals:
    faces = [
        [hex_nodes[0], hex_nodes[1], hex_nodes[2], hex_nodes[3]],  # bottom face
        [hex_nodes[4], hex_nodes[5], hex_nodes[6], hex_nodes[7]],  # top face
        [hex_nodes[0], hex_nodes[1], hex_nodes[5], hex_nodes[4]],  # side face 1
        [hex_nodes[1], hex_nodes[2], hex_nodes[6], hex_nodes[5]],  # side face 2
        [hex_nodes[2], hex_nodes[3], hex_nodes[7], hex_nodes[6]],  # side face 3
        [hex_nodes[3], hex_nodes[0], hex_nodes[4], hex_nodes[7]]   # side face 4
    ]
    triangles = []
    for quad in faces:
        # Split each quadrilateral into two triangles
        triangles.append([quad[0], quad[1], quad[2]])
        triangles.append([quad[0], quad[2], quad[3]])
    return triangles

In [51]:
nodes_dict = {node_entry['node']: (node_entry['x'], node_entry['y'], node_entry['z']) for node_entry in group1}

print(nodes_dict)

{1: (17.00863692561619, -73.39911768987086, 1637.73381829317), 2: (17.58661833875242, -83.34197270722693, 1637.74293133895), 3: (16.86201315934084, -93.3304702070233, 1637.288171307049), 4: (12.34375228837095, -70.22061574892321, 1638.402355392379), 5: (11.25882546160152, -83.70084813210717, 1639.046504717372), 6: (12.26452209831464, -97.03602966073136, 1637.994574903735), 7: (6.013261376149083, -68.12180024878165, 1638.944291608482), 8: (4.794845498846031, -83.97915418212418, 1639.763303235005), 9: (6.028876785706972, -99.73281924010388, 1638.727771488912), 10: (-1.765245177496509, -67.46282993270667, 1639.049526676457), 11: (-1.727437039894693, -84.1627883327494, 1639.950740861956), 12: (-1.687954701850093, -100.8592978903223, 1639.088977088665), 13: (-9.525337098030974, -68.43985726140492, 1638.523671675043), 14: (-8.24001683841247, -84.2551978308335, 1639.606824152404), 15: (-9.401628968498938, -100.0856802436372, 1638.758905283395), 16: (-15.8117798276047, -70.79179567514953, 1637

In [43]:
# Convert all elements into triangles.
all_triangles = []
for elem in elements:
    hex_nodes = elem['nodes']
    # Convert the hexahedron into a list of triangles.
    triangles = hexahedron_to_triangles(hex_nodes)
    all_triangles.extend(triangles)

In [45]:
len(all_triangles)

15312

In [46]:
# --- Step 3: Prepare the Triangle Indices for Plotly's Mesh3d ---
# Plotly requires the triangle vertex indices (i, j, k) to be zero-indexed.
i_vals = []
j_vals = []
k_vals = []
for tri in all_triangles:
    # Subtract 1 from each node ID to convert to zero-indexing.
    i_vals.append(tri[0] - 1)
    j_vals.append(tri[1] - 1)
    k_vals.append(tri[2] - 1)

In [52]:
# --- Step 4: Prepare Node Coordinates ---
# Build lists for the x, y, z coordinates. We assume the node IDs in nodes_dict are consecutive.
num_nodes = max(nodes_dict.keys())
x_all = [nodes_dict[i][0] for i in range(1, num_nodes+1)]
y_all = [nodes_dict[i][1] for i in range(1, num_nodes+1)]
z_all = [nodes_dict[i][2] for i in range(1, num_nodes+1)]

In [55]:
len(x_all)

1637

In [72]:
# Create a Mesh3d plot with the parsed coordinates and connectivity.
mesh = go.Mesh3d(
    x=x_all,
    y=y_all,
    z=z_all,
    i=i_vals,
    j=j_vals,
    k=k_vals,
    opacity=0.5,
    color='lightblue'
)

fig = go.Figure(data=[mesh])
fig.update_layout(
    title="3D Mesh using Connectivity from exf File",
    scene=dict(
        xaxis=dict(title='X', range=axis_range),
        yaxis=dict(title='Y', range=axis_range),
        zaxis=dict(title='Z', range=[range + 800 for range in axis_range]),
        aspectmode="cube"
    ),
    width=800,
    height=800
)
fig.show()

In [73]:
# Extract coordinate lists for plotting
x_vals = [node['x'] for node in nodes]
y_vals = [node['y'] for node in nodes]
z_vals = [node['z'] for node in nodes]
scatter = go.Scatter3d(
    x=x_vals,
    y=y_vals,
    z=z_vals,
    mode='markers',
    marker=dict(size=2, color="#eae0c8")
)

fig = go.Figure(data=[mesh, scatter])
fig.update_layout(
    title="3D Mesh using Connectivity from exf File",
    scene=dict(
        xaxis=dict(title='X', range=axis_range,
                   backgroundcolor="black",
                   gridcolor="gray",
                   showbackground=True,
                   zerolinecolor="gray"),
        yaxis=dict(title='Y', range=axis_range,
                   backgroundcolor="black",
                   gridcolor="gray",
                   showbackground=True,
                   zerolinecolor="gray"),
        zaxis=dict(title='Z', range=[range + 800 for range in axis_range],
                   backgroundcolor="black",
                   gridcolor="gray",
                   showbackground=True,
                   zerolinecolor="gray"),
        aspectmode="cube"
    ),
    paper_bgcolor="black",  # background color outside the 3D scene
    plot_bgcolor="black",   # background color of the plotting area
    title_font_color="white",  # make the title text white
    font=dict(color="white"),
    width=800,
    height=800
)
fig.show()

# Save as csv

In [75]:
import pandas as pd

In [81]:
node_df_all = pd.DataFrame(nodes).set_index("node")
node_df_all

,x,y,z
node,,,
1,17.008637,-73.399118,1637.733818
2,17.586618,-83.341973,1637.742931
3,16.862013,-93.330470,1637.288171
4,12.343752,-70.220616,1638.402355
5,11.258825,-83.700848,1639.046505
...,...,...,...
13397,8.309809,-209.589401,1017.513123
13398,-8.836831,-200.259171,972.901550
13399,10.332440,-201.513947,973.711731


In [82]:
node_df_group1 = pd.DataFrame(group1).set_index("node")
node_df_group1

,x,y,z
node,,,
1,17.008637,-73.399118,1637.733818
2,17.586618,-83.341973,1637.742931
3,16.862013,-93.330470,1637.288171
4,12.343752,-70.220616,1638.402355
5,11.258825,-83.700848,1639.046505
...,...,...,...
1633,-168.466737,-228.708649,-65.839073
1634,-150.088538,-233.628756,-69.180170
1635,-131.718210,-239.220402,-71.725331


In [83]:
node_df_group2 = pd.DataFrame(group2).set_index("node")
node_df_group2

,x,y,z
node,,,
11012,14.700808,-84.992948,1351.466863
11013,42.820559,-41.129769,1129.988292
11014,20.679611,-112.246951,1179.096599
11015,87.686166,-185.636391,1144.795592
11016,-49.377613,-47.892392,1124.340410
...,...,...,...
13397,8.309809,-209.589401,1017.513123
13398,-8.836831,-200.259171,972.901550
13399,10.332440,-201.513947,973.711731


In [93]:
mesh_df_group1 = pd.DataFrame({"triangle": list(range(len(i_vals))),
                               "i": i_vals,
                               "j": j_vals,
                               "k": k_vals}).set_index("triangle")
mesh_df_group1

,i,j,k
triangle,,,
0,0,3,53
1,0,53,56
2,1,4,54
3,1,54,57
4,0,3,4
...,...,...,...
15307,1574,1636,1586
15308,1624,1613,1625
15309,1624,1625,1636
